In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.precision", 3)

CSV_PATH = r"./magic.csv"

df = pd.read_csv(CSV_PATH)
df.columns

Index(['routablemask_link', 'routablemask_link/_text',
       'routablemask_link/_title', 'routablemask_link_numbers',
       'routablemask_content', 'imagebadge_value', 'imagebadge_value_numbers',
       'routable_link', 'routable_link/_text', 'routable_link/_title',
       'routable_link_numbers', 'listingtype_value', 'pricelarge_value',
       'pricelarge_value_prices', 'propertyinfo_value',
       'propertyinfo_value_numbers', 'fineprint_value',
       'fineprint_value_numbers', 'tozcount_number', 'tozfresh_value',
       'tablegrouped_values', 'tablegrouped_values_prices', '_PAGE_NUMBER'],
      dtype='object')

In [3]:
df.head().T

,0,1,2,3,4
routablemask_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routablemask_link/_text,5 photos,10 photos,NaN,9 photos,5 photos
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,5.0,10.0,NaN,9.0,5.0
routablemask_content,NaN,NaN,NaN,NaN,NaN
imagebadge_value,5 photos,10 photos,NaN,9 photos,5 photos
imagebadge_value_numbers,5.0,10.0,NaN,9.0,5.0
routable_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routable_link/_text,2 Ellwood St,603 W 140th St,"9 E 129th St # 1, New York, NY10035","710 Riverside Dr APT 2C, New York, NY10031","413 E 84th St APT 8, New York, NY10028"
routable_link/_title,"2 Ellwood St APT 5H, New York, NY Real Estate","603 W 140th St APT 44, New York, NY Real Estate","9 E 129th St # 1, New York, NY Real Estate","710 Riverside Dr APT 2C, New York, NY Real Estate","413 E 84th St APT 8, New York, NY Real Estate"


In [4]:
mu = df[df['listingtype_value'].str.contains('Apartments For')]
su = df[df['listingtype_value'].str.contains('Apartment For')]

In [5]:
su['propertyinfo_value']

2                    1 bd • 1 ba
3      2 bds • 2 ba • 1,016 sqft
4                  Studio • 1 ba
5                   2 bds • 1 ba
7                   2 bds • 1 ba
                 ...            
490                  1 bd • 1 ba
492                  1 bd • 1 ba
496                Studio • 1 ba
497     Studio • 1 ba • 550 sqft
499                  1 bd • 1 ba
Name: propertyinfo_value, Length: 339, dtype: object

In [6]:
len(su[~(su['propertyinfo_value'].str.contains('Studio') | su['propertyinfo_value'].str.contains('bd'))])

0

In [7]:
len(su[~(su['propertyinfo_value'].str.contains('bd'))])

122

In [8]:
su[~(su['propertyinfo_value'].str.contains('ba'))]['propertyinfo_value']

84     Studio
106      1 bd
150    Studio
225      1 bd
313    Studio
331      1 bd
Name: propertyinfo_value, dtype: object

In [9]:
no_baths = su[~(su['propertyinfo_value'].str.contains('ba'))]

In [10]:
sucln = su[~su.index.isin(no_baths.index)]

In [12]:
def parse_info(row):
  if not 'sqft' in row:
    br, ba = row.split('•')[:2]
    sqft = np.nan
  else:
    br, ba, sqft = row.split('•')[:3]
  return pd.Series({'Beds': br, 'Baths':ba, 'Sqrt':sqft})

attr = sucln['propertyinfo_value'].apply(parse_info)

In [13]:
att_cln = attr.applymap(lambda x: x.strip().split(' ')[0] if isinstance (x, str) else np.nan )

In [14]:
sujud = sucln.join(att_cln)

# parse out zip, floor

In [16]:
def parse_addy(r):
  so_zip = re.search(', NY(\d+)', r)
  so_flr = re.search('(?:APT|#)\s+(\d+)[A-Z]+,', r)
  if so_zip:
    zipc = so_zip.group(1)
  else:
    zipc = np.nan
  
  if so_flr:
    flr = so_flr.group(1)
  else:
    flr = np.nan
  return pd.Series({'Zip':zipc, 'Floor':flr})

flrzip = sujud['routable_link/_text'].apply(parse_addy)

suf = sujud.join(flrzip)



In [31]:
# np.isnan(np.array(suf['Zip']))
array1 = np.array([1,2,3,np.nan, np.nan])
np.isnan(array1)
array1

array([ 1.,  2.,  3., nan, nan])

In [33]:
sudf = suf[['pricelarge_value_prices','Beds','Baths','Sqrt','Floor','Zip']]

In [35]:
sudf.rename(columns={
  'pricelarge_value_prices':'Rent'
},inplace = True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\2600392517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.rename(columns={


In [36]:
sudf.reset_index(drop=True, inplace=True)

In [37]:
sudf

,Rent,Beds,Baths,Sqrt,Floor,Zip
0,1750.0,1,1,NaN,NaN,10035
1,3000.0,2,2,"1,016",2,10031
2,2300.0,Studio,1,NaN,NaN,10028
3,2500.0,2,1,NaN,6,10035
4,2800.0,2,1,NaN,NaN,10012
...,...,...,...,...,...,...
328,2265.0,1,1,NaN,NaN,10021
329,2150.0,1,1,NaN,4,10028
330,2195.0,Studio,1,NaN,3,10019
331,1795.0,Studio,1,550,NaN,10035


In [38]:
sudf.describe()

,Rent
count,333.000
mean,2492.628
std,366.882
min,1500.000
25%,2200.000
50%,2525.000
75%,2800.000
max,3000.000


In [39]:
sudf.loc[:,'Beds'] = sudf['Beds'].map(lambda x: 0 if 'Studio' in x else x)

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\938725536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.loc[:,'Beds'] = sudf['Beds'].map(lambda x: 0 if 'Studio' in x else x)


In [40]:
sudf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rent    333 non-null    float64
 1   Beds    333 non-null    object 
 2   Baths   333 non-null    object 
 3   Sqrt    108 non-null    object 
 4   Floor   164 non-null    object 
 5   Zip     320 non-null    object 
dtypes: float64(1), object(5)
memory usage: 15.7+ KB


In [41]:
sudf.loc[:, 'Rent'] = sudf.loc[:, 'Rent'].astype(int)
sudf.loc[:, 'Beds'] = sudf.loc[:, 'Beds'].astype(int)

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\3602357540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.loc[:, 'Rent'] = sudf.loc[:, 'Rent'].astype(int)
C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\3602357540.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sudf.loc[:, 'Rent'] = sudf.loc[:, 'Rent'].astype(int)
C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\3602357540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [42]:
sudf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rent    333 non-null    int32 
 1   Beds    333 non-null    int32 
 2   Baths   333 non-null    object
 3   Sqrt    108 non-null    object
 4   Floor   164 non-null    object
 5   Zip     320 non-null    object
dtypes: int32(2), object(4)
memory usage: 13.1+ KB


In [44]:
sudf.loc[:, 'Baths'] = sudf.loc[:, 'Baths'].astype(float)

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\1102597879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.loc[:, 'Baths'] = sudf.loc[:, 'Baths'].astype(float)
C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\1102597879.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sudf.loc[:, 'Baths'] = sudf.loc[:, 'Baths'].astype(float)


In [45]:
sudf.loc[:, 'Sqrt'] = sudf.loc[:, 'Sqrt'].str.replace(',','')

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\1239239243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.loc[:, 'Sqrt'] = sudf.loc[:, 'Sqrt'].str.replace(',','')


In [46]:
sudf.loc[:, 'Sqrt'] = sudf.loc[:, 'Sqrt'].astype(float)

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\2550817120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.loc[:, 'Sqrt'] = sudf.loc[:, 'Sqrt'].astype(float)
C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\2550817120.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sudf.loc[:, 'Sqrt'] = sudf.loc[:, 'Sqrt'].astype(float)


In [47]:
sudf.loc[:, 'Floor'] = sudf.loc[:, 'Floor'].astype(float)

C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\2248339159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sudf.loc[:, 'Floor'] = sudf.loc[:, 'Floor'].astype(float)
C:\Users\ACER\AppData\Local\Temp\ipykernel_23036\2248339159.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sudf.loc[:, 'Floor'] = sudf.loc[:, 'Floor'].astype(float)


In [48]:
sudf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rent    333 non-null    int32  
 1   Beds    333 non-null    int32  
 2   Baths   333 non-null    float64
 3   Sqrt    108 non-null    float64
 4   Floor   164 non-null    float64
 5   Zip     320 non-null    object 
dtypes: float64(3), int32(2), object(1)
memory usage: 13.1+ KB


In [49]:
sudf.describe()

,Rent,Beds,Baths,Sqrt,Floor
count,333.000,333.000,333.000,108.000,164.000
mean,2492.628,0.820,1.008,528.981,11.201
std,366.882,0.718,0.082,133.050,86.178
min,1500.000,0.000,1.000,280.000,1.000
25%,2200.000,0.000,1.000,447.500,2.000
50%,2525.000,1.000,1.000,512.000,4.000
75%,2800.000,1.000,1.000,600.000,5.000
max,3000.000,3.000,2.000,1090.000,1107.000
